In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import os.path
import sys
import time

from six.moves import xrange  
import tensorflow as tf
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from scipy.stats import pearsonr

from src.parser import gen_parser

FLAGS = None

parser = gen_parser()

FLAGS, unparsed = parser.parse_known_args()


/home/ola-sammy/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Setting Up Savings and Files IDs

In [2]:
IDtag = str(int(time.time())) # use a unix time ID for each training
working_dir = os.getcwd()
print("Working dir: " + working_dir)
#mansave_dir = '/home/ola-sammy/Documents/Imbizo2019/Project/V1_predictor_Sammy/manualsave'
mansave_dir = working_dir + '/manualsave' # the directory to save to
print("manual save dir: " + mansave_dir)
savedirname = mansave_dir + '/training_manualsave_' + IDtag #filename for saving training
savenetworkname = mansave_dir + '/network_manualsave_' + IDtag # saving network parameters
savefigname = mansave_dir + '/training_plot_' + IDtag # saving training figure
neuronbessavedirt = mansave_dir + '/neuronloss_' + IDtag # not used 
pearsonsavedir = mansave_dir + '/personplot' + IDtag # saving performance figure 
pearsonsavedatadir = mansave_dir + '/persondata' + IDtag # saving performance data 
losssavedir = mansave_dir + '/lossplot' + IDtag # not used


Working dir: /home/ola-sammy/Documents/Imbizo2019/Project/V1_predictor_Sammy
manual save dir: /home/ola-sammy/Documents/Imbizo2019/Project/V1_predictor_Sammy/manualsave


#### Making a Folder to Save to

In [3]:
if (tf.gfile.Exists(mansave_dir) == 0):
    tf.gfile.MakeDirs(mansave_dir)

#### Import the already Built Network: build_network

In [4]:
import src.build_network as build_network
from src.build_network import simpleRNN, ConvNetDrop, RConvNet, LnonL
from src.utils import DataLoader

#### List of data to be fed into the Network.

In [5]:
data_dir = FLAGS.data_dir
fileindex = FLAGS.fileindex
#Note: Files (Data)  must be in the same directory.
filearray = [
        '01_process.mat',
        '02_process.mat',
        '03_process.mat',
        '04_process.mat',
        '05_process.mat',
        '06_process.mat',
        '07_process.mat',
        '08_process.mat',
        '09_process.mat',
        '10_process.mat']

filename = filearray[fileindex]
filepath = os.path.join(data_dir,filename)

data = DataLoader([filepath],FLAGS) # loads and formats all the data


loading 1 file(s)
/home/ola-sammy/Documents/Imbizo2019/Project/V1_predictor_Sammy/data/02_process.mat
['/home/ola-sammy/Documents/Imbizo2019/Project/V1_predictor_Sammy/data/02_process.mat ncells:']
37


In [6]:
print("total number of cells is %d" % (data.numcell))
print("y eval shape")
print(np.shape(data.yeval))
evalvar = np.var(data.yeval,axis = 0)
print("eval var is:") 
print(np.mean(evalvar))

total number of cells is 37
y eval shape
(52, 37)
eval var is:
0.1966469344714537


In [7]:
numtrain = data.numtrain
numearlystop = data.numearlystop
numeval =  data.numeval 
numbatch = np.minimum(numtrain,FLAGS.batch_size)
numtrials = data.numtrials
numpixx = data.numpixx  
numpixy = data.numpixy
print('number of trials %d' % (numtrials))
print('number of  pixels are %d X %d' % (numpixx,numpixy))


number of trials 270
number of  pixels are 33 X 33


### Function to Train the model

In [8]:
def run_training(lossbaseline, lossbaselineneuron, model = None, dataset = None):
    if dataset is not None:
        data = dataset

    #start the training
    with tf.Graph().as_default():
    # generate placeholders
        images_placeholder = tf.placeholder(tf.float32, shape = (None, data.numpixx, data.numpixy, 1))
        activity_placeholder = tf.placeholder(tf.float32, shape = (None,data.numcell))
        keep_prob_placeholder = tf.placeholder(tf.float32)
        baselineloss_placeholder = tf.placeholder(tf.float32, shape=(data.numcell))

        # network hyper-parameters as arrays
        #2 conv layers
        if FLAGS.numconvlayer == 1:
            num_filter_list = [FLAGS.conv1] 
            filter_size_list = [FLAGS.conv1size] 
            pool_stride_list = [FLAGS.nstride1] 
            pool_k_list =[FLAGS.nk1]
        elif FLAGS.numconvlayer == 2:
            num_filter_list = [FLAGS.conv1, FLAGS.conv2] # [16,32]
            filter_size_list = [FLAGS.conv1size, FLAGS.conv2size] # [7,7]
            pool_stride_list = [FLAGS.nstride1, FLAGS.nstride2] # [2,2]
            pool_k_list =[FLAGS.nk1, FLAGS.nk2 ]  # [3, 3]
        elif FLAGS.numconvlayer == 3:
            num_filter_list = [FLAGS.conv1, FLAGS.conv2, FLAGS.conv2] # [16,32]
            filter_size_list = [FLAGS.conv1size, FLAGS.conv2size, FLAGS.conv2size] # [7,7]
            pool_stride_list = [FLAGS.nstride1, FLAGS.nstride2, FLAGS.nstride2] # [2,2]
            pool_k_list =[FLAGS.nk1, FLAGS.nk2, FLAGS.nk2]  # [3, 3]

        #1 all-to-all hidden layer
        dense_list = [FLAGS.hidden1] # [300]
        keep_prob = FLAGS.dropout # 0.55
        numcell = data.numcell

        # if no model passed, use default
        if model is None:

            model = ConvNetDrop(images_placeholder, 
                num_filter_list, filter_size_list, pool_stride_list, 
                pool_k_list, dense_list, keep_prob_placeholder,numcell)

        else:
            model(images_placeholder)

        print("model shape is")
        print(model.output.get_shape())

        ## Add to the Graph the Ops for loss calculation.
        loss = build_network.loss(model.output, activity_placeholder)
        loss_per_neuron = build_network.losspercell(model.output, activity_placeholder)
        train_op = build_network.training(loss, FLAGS.learning_rate)

        # Add the variable initializer Op.
        init = tf.global_variables_initializer()

        # Create a session for running Ops on the Graph.
        sess = tf.Session()
        # Run the Op to initializactivitytraine the variables.
        sess.run(init)

        # setting up recording training progress
        steplist = []
        evallist = []
        trainlist = []
        earlystoplist = []
        rmeanlist = []
        rcelllist = []
    
        ## Start the training loop.
        lossearlystopmin = 10e6 # large dummy value use for finding the minimum loss in the early stop data
        for step in xrange(FLAGS.max_steps):
            start_time = time.time()
            batchindex = np.random.permutation(numtrain)[0:numbatch]
            xtrainbatch = data.xtrain[batchindex ,:,:,:]
            ytrainbatch = data.ytrain[batchindex ,:]
      
            feed_dict = {
                images_placeholder: xtrainbatch,
                activity_placeholder: ytrainbatch,
                keep_prob_placeholder: keep_prob
                }

            _, loss_value = sess.run([train_op, loss],
                               feed_dict=feed_dict)

            FVE = 1 - loss_value / lossbaseline
            duration = time.time() - start_time
            # print progress.
            if step % 50 == 0:
                ## Print status

                print('Step %d: loss = %.4f; FVE = %5.2f (%.3f sec)' % (step, loss_value, FVE, duration))
                  ## save and evaluate the model 
            if (step - 1) % 200 == 0 or (step + 1) == FLAGS.max_steps or step == 1:
                ## evaluate and save progress 

                steplist.append(step) # list of training steps
                ## Evaluate against the training set.
                feed_dict={images_placeholder: data.xtrain, activity_placeholder: data.ytrain, keep_prob_placeholder:1}
                losstrain = sess.run(loss, feed_dict=feed_dict)
                trainlist.append(losstrain) # list of loss on training set
                ## Evaluate against the eval set.

                feed_dict={images_placeholder: data.xeval, activity_placeholder: data.yeval, keep_prob_placeholder:1}
                losseval = sess.run(loss, feed_dict=feed_dict)

          
                evallist.append(losseval) # list of loss on eval set
                #computting r eval on eval set
                actpredict_eval = sess.run(model.output, feed_dict=feed_dict)
                reval= np.zeros(data.numcell)
                for icell in range(data.numcell):
                    reval[icell], peval = pearsonr(actpredict_eval[:,icell], data.yeval[:,icell])
                    if np.isnan(reval[icell]):
                        reval[icell] = 0
                rcelllist.append(reval) # list of r eval on eval set
                revalmean = np.mean(reval)
                rmeanlist.append(revalmean) # list of  mean r eval on eval set
                ## Evaluate againts early stop data
                feed_dict={images_placeholder: data.xstop, activity_placeholder: data.ystop, keep_prob_placeholder:1}
                lossearlystop = sess.run(loss, feed_dict=feed_dict)
                earlystoplist.append(lossearlystop) # list of loss on early stop set

                ## plot and save training
                if FLAGS.savetraining:
                    mansavefig(trainlist, earlystoplist, evallist, rmeanlist, steplist, lossbaseline)

                ## Finding the minumum loss on the early stop data set
                ## check if new early stop min. If so, treat as best trained nextwork 
                if lossearlystop < lossearlystopmin:  # check if early stop is new min 
                    lossearlystopmin =  lossearlystop # save loss as new minumum loss on the early stop data set
                    FVEeval = 1 - losseval / lossbaseline
                    lossevalmin = losseval
                    print("early stop")
                    print("Performance is:")
                    print('squared (loss) = %.4f; FVE = %5.3f' % (losseval,FVEeval))

                    # compute r val again. Could use above values. 
                    rval= np.zeros(data.numcell)
                    feed_dict={images_placeholder: data.xeval, activity_placeholder: data.yeval, keep_prob_placeholder:1}
                    loss_per_neuron_eval = sess.run(loss_per_neuron, feed_dict=feed_dict)
                    actpredict_eval = sess.run(model.output, feed_dict=feed_dict)
                    rval= np.zeros(data.numcell)
                    for icell in range(data.numcell):
                        rval[icell], pval = pearsonr(actpredict_eval[:,icell], data.yeval[:,icell])
                        if np.isnan(rval[icell]):
                            rval[icell] = 0
                    rmean = np.mean(rval)
                    print('r = %5.3f' % (rmean))
                    print("More training!")
                    if FLAGS.savenetwork:
                        network_save(step) #save the parameters of network
                    if FLAGS.save:
                        plotandsaver(rval, step, loss_per_neuron_eval, lossbaselineneuron) #save the performance of network 

        print("Final results")
        print("Best Performance: ")
        print('r = %5.3f +- %5.3f;  squared (loss) = %.4f;  FVE = %5.3f' % (rmean, np.std(rval)/np.sqrt(data.numcell), losseval, FVEeval))
        print("Evaluation Variance is:") 
        print(np.mean(evalvar)) 
        rerror = np.std(rval)/np.sqrt(data.numcell)   
        

In [9]:
def baseline_error(): 
    # make the baseline predictor
    
    meanactdum  = np.mean(data.ytrain, axis=0)

    meanact =  np.reshape(meanactdum, [1, meanactdum.size])
    meanpredict = np.repeat(meanact, data.numeval, axis=0)

    #make placeholdfers for baseline 
    y_ = tf.placeholder(tf.float32, shape = (None, data.numcell))
    meanpredict_placeholder = tf.placeholder(tf.float32, shape=(None,data.numcell))
    loss_baseline = build_network.loss(meanpredict_placeholder,y_)
    loss_baseline_percell = build_network.losspercell(meanpredict_placeholder,y_)
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    # Evaluate the baseline.
    feed_dict={meanpredict_placeholder: meanpredict, y_: data.yeval}
    loss_baseline_eval = sess.run(loss_baseline, feed_dict=feed_dict)
    loss_baseline_eval_percell = sess.run(loss_baseline_percell, feed_dict=feed_dict)
    #print and return baseline loss
    print('')
    print('Eval data baseline loss = %.4f' % (loss_baseline_eval))
    print("variance of evaluation")
    vare = np.var(data.yeval, axis = 0)
    print(np.shape(vare) )
    print(np.mean(vare))
    manvare = np.mean(np.mean(np.square(meanpredict-data.yeval)))
    print("man calc variance/loss of evaluation")
    print(manvare)
    return loss_baseline_eval, loss_baseline_eval_percell

In [10]:
def plotandsaver(rval,step, loss_per_neuron_eval, lossbaselineneuron):
    figN, axN = plt.subplots()
    bar_width = 0.40
    xvals = np.arange(len(rval))+1-bar_width/2
    axN.bar(xvals, rval)
    axN.set_xlabel('cell', color='k')
    axN.set_ylabel('r', color='k')
    plt.savefig(pearsonsavedir)
    traintrials = data.traintrials
    earlystoptrials = data.earlystoptrials
    evaltrials = data.evaltrials
    np.save(pearsonsavedatadir,[rval,step,loss_per_neuron_eval,lossbaselineneuron, evalvar,traintrials,earlystoptrials,evaltrials,FLAGS])
    plt.close(figN)
 

In [11]:
def network_save(step):
    print('saving network as: ' + savenetworkname)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    #manually get all of the network parameters
    with tf.variable_scope('conv1', reuse = True) as scope:
        weights1 = tf.get_variable('W')
        WC1 = sess.run(weights1)
        biases1 = tf.get_variable('b')
        BC1 =sess.run(biases1)
    if FLAGS.numconvlayer != 1:
        with tf.variable_scope('conv2', reuse = True) as scope:
            weights2 = tf.get_variable('W')
            WC2 = sess.run(weights2)
            biases2 = tf.get_variable('b')
            BC2 =sess.run(biases2)
    with tf.variable_scope('dense1', reuse = True) as scope:
        weights3 = tf.get_variable('W')
        WH = sess.run(weights3)
        biases3 = tf.get_variable('b')
        BH = sess.run(biases3)
    with tf.variable_scope('linear', reuse = True) as scope:
        weights4 = tf.get_variable('W')
        WL = sess.run(weights4)
        biases4 = tf.get_variable('b')
        BL = sess.run(biases4)
    if FLAGS.numconvlayer == 1:
        np.save(savenetworkname,[WC1, BC1, WH, BH, WL, BL, step])
    else:
        np.save(savenetworkname,[WC1, BC1, WC2, BC2, WH, BH, WL, BL, step])


In [12]:
def mansavefig(trainlist, earlystoplist, evallist, rlist, step, lossbaseline):
    plt.ioff()
    minindex = np.argmin(np.asarray(earlystoplist))
    xrange = max(step)
    
    textx = 100 + xrange/8
    if (textx<step[minindex]) & (step[minindex]<xrange/2):
        textx = xrange/12 + step[minindex]
    ymax = 2.0*lossbaseline
    ymin = -0.1*lossbaseline
    deltay = ymax - ymin
    figA, axA = plt.subplots()

    axA.plot(np.asarray(step), np.asarray(trainlist), 'r', label='training')
    axA.plot(np.asarray(step), np.asarray(earlystoplist), 'r--', label = 'early stop')
    axA.plot(np.asarray(step), np.asarray(evallist), 'r:', label = 'evaluation')
    axA.set_ylim([ymin,ymax])
    axA.set_xlabel('step')
      ## Make the y-axis label, ticks and tick labels match the line color.
    axA.axvline(x = step[minindex], ymin = -0.1, ymax = 2, linewidth = 2, color = 'k')
    axA.set_ylabel('least squared loss', color = 'k')
    axA.tick_params('y', colors = 'k')
    axA.annotate('Step: %d' % (step[minindex] ), xy = (textx, 1.0*deltay+ymin))
    axA.annotate('avg r: %.3f; eval loss %.4f / eval var %.4f' % (np.mean(rlist[minindex]),np.mean(evallist[minindex]), np.mean(evalvar) ), xy=(textx, 0.95*deltay+ymin))
    axA.annotate('Max pool 1 stride: %d ' % (FLAGS.nstride1), xy=(textx, 0.90*deltay+ymin))
    axA.annotate('Max pool 1 kernel size: %d ' % (FLAGS.nk1), xy=(textx, 0.85*deltay+ymin))
    axA.annotate('Conv 1 size: %d X %d ' % (FLAGS.conv1size, FLAGS.conv1size), xy=(textx, 0.80*deltay+ymin))
    axA.annotate('number of L 1 filters: %d' % (FLAGS.conv1), xy=(textx, 0.75*deltay+ymin))
    axA.annotate('Max pool 2 stride: %d ' % (FLAGS.nstride2), xy=(textx, 0.70*deltay+ymin))
    axA.annotate('Max pool 2 kernel size: %d ' % (FLAGS.nk2), xy=(textx, 0.65*deltay+ymin))
    axA.annotate('Conv 2 size: %d X %d ' % (FLAGS.conv2size, FLAGS.conv2size), xy=(textx, 0.60*deltay+ymin))
    axA.annotate('number of L 2 filters: %d' % (FLAGS.conv2), xy=(textx, 0.55*deltay+ymin))
    axA.annotate('Dropout keep rate: %.3f ' % (FLAGS.dropout), xy=(textx, 0.50*deltay+ymin))
    axA.annotate('%d hidden elements' % (FLAGS.hidden1 ), xy=(textx, 0.45*deltay+ymin))
    axA.annotate('Learning rate: %.1e' % (FLAGS.learning_rate), xy=(textx, 0.40*deltay+ymin))
    axA.annotate('Num conv layers: %d ' % (FLAGS.numconvlayer), xy=(textx, 0.35*deltay+ymin))
    
    plt.legend(loc = 'best')
    axB = axA.twinx()
    axB.plot(np.asarray(step), np.asarray(rlist), 'k')
    axB.set_ylim([-0.1,1])
    axB.set_ylabel('r', color = 'k')
    axB.tick_params('y', colors = 'k')
    figA.tight_layout()
    plt.savefig(savefigname)
    plt.close(figA)

    ##saving training parameters
    traintrials = data.traintrials
    earlystoptrials = data.earlystoptrials
    evaltrials = data.evaltrials
    np.save(savedirname,[FLAGS, trainlist, earlystoplist, evallist, rlist,
        step, lossbaseline, traintrials, earlystoptrials, evaltrials])

In [13]:
def main(_):
  lossbaseline, lossbaselineneuron  = baseline_error()
  run_training(lossbaseline, lossbaselineneuron,dataset = data)

# run main
if __name__ == '__main__':
    tf.app.run(main = main, argv=[sys.argv[0]] + unparsed)


Eval data baseline loss = 0.2007
variance of evaluation
(37,)
0.1966469344714537
man calc variance/loss of evaluation
0.20070449167115834
start building network
building layer conv1
with 16 filters shaped 
[3, 3, 1, 16]
curret layer is 
Tensor("conv1/pool:0", shape=(?, 15, 15, 16), dtype=float32)
building layer conv2
with 32 filters shaped 
[3, 3, 16, 32]
curret layer is 
Tensor("conv2/pool:0", shape=(?, 6, 6, 32), dtype=float32)
building layer dense1
maping 1152 elemements to 300 elements
building linear layer
linear maping 300 elemements to 37 elements
Finished building network
model shape is
(?, 37)
Step 0: loss = 0.7946; FVE = -2.96 (1.348 sec)


/home/ola-sammy/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


early stop
Performance is:
squared (loss) = 0.7587; FVE = -2.780
r = 0.033
More training!
Step 50: loss = 0.4960; FVE = -1.47 (0.022 sec)
Step 100: loss = 0.3413; FVE = -0.70 (0.028 sec)
Step 150: loss = 0.3527; FVE = -0.76 (0.029 sec)
Step 200: loss = 0.2574; FVE = -0.28 (0.029 sec)
early stop
Performance is:
squared (loss) = 0.2095; FVE = -0.044
r = 0.085
More training!
Step 250: loss = 0.2454; FVE = -0.22 (0.023 sec)
Step 300: loss = 0.2557; FVE = -0.27 (0.024 sec)
Step 350: loss = 0.2463; FVE = -0.23 (0.024 sec)
Step 400: loss = 0.2381; FVE = -0.19 (0.034 sec)
early stop
Performance is:
squared (loss) = 0.1965; FVE = 0.021
r = 0.135
More training!
Step 450: loss = 0.2399; FVE = -0.20 (0.023 sec)
Step 500: loss = 0.2341; FVE = -0.17 (0.029 sec)
Step 550: loss = 0.2352; FVE = -0.17 (0.029 sec)
Step 600: loss = 0.2065; FVE = -0.03 (0.022 sec)
early stop
Performance is:
squared (loss) = 0.1850; FVE = 0.078
r = 0.204
More training!
Step 650: loss = 0.2063; FVE = -0.03 (0.025 sec)
Step 7

Step 5700: loss = 0.1251; FVE =  0.38 (0.024 sec)
Step 5750: loss = 0.1310; FVE =  0.35 (0.024 sec)
Step 5800: loss = 0.1251; FVE =  0.38 (0.026 sec)
Step 5850: loss = 0.1290; FVE =  0.36 (0.026 sec)
Step 5900: loss = 0.1158; FVE =  0.42 (0.022 sec)
Step 5950: loss = 0.1154; FVE =  0.43 (0.036 sec)
Step 6000: loss = 0.1205; FVE =  0.40 (0.024 sec)
early stop
Performance is:
squared (loss) = 0.1264; FVE = 0.370
r = 0.545
More training!
Step 6050: loss = 0.1260; FVE =  0.37 (0.021 sec)
Step 6100: loss = 0.1202; FVE =  0.40 (0.021 sec)
Step 6150: loss = 0.1301; FVE =  0.35 (0.022 sec)
Step 6200: loss = 0.1081; FVE =  0.46 (0.020 sec)
early stop
Performance is:
squared (loss) = 0.1258; FVE = 0.373
r = 0.547
More training!
Step 6250: loss = 0.1186; FVE =  0.41 (0.025 sec)
Step 6300: loss = 0.1282; FVE =  0.36 (0.023 sec)
Step 6350: loss = 0.1109; FVE =  0.45 (0.024 sec)
Step 6400: loss = 0.1168; FVE =  0.42 (0.021 sec)
early stop
Performance is:
squared (loss) = 0.1246; FVE = 0.379
r = 0.54

early stop
Performance is:
squared (loss) = 0.1099; FVE = 0.452
r = 0.593
More training!
Step 12450: loss = 0.0877; FVE =  0.56 (0.022 sec)
Step 12500: loss = 0.0958; FVE =  0.52 (0.024 sec)
Step 12550: loss = 0.0912; FVE =  0.55 (0.040 sec)
Step 12600: loss = 0.1007; FVE =  0.50 (0.021 sec)
Step 12650: loss = 0.0917; FVE =  0.54 (0.023 sec)
Step 12700: loss = 0.0957; FVE =  0.52 (0.021 sec)
Step 12750: loss = 0.0842; FVE =  0.58 (0.021 sec)
Step 12800: loss = 0.0887; FVE =  0.56 (0.022 sec)
Step 12850: loss = 0.0984; FVE =  0.51 (0.022 sec)
Step 12900: loss = 0.0851; FVE =  0.58 (0.023 sec)
Step 12950: loss = 0.0907; FVE =  0.55 (0.026 sec)
Step 13000: loss = 0.0905; FVE =  0.55 (0.020 sec)
Step 13050: loss = 0.0886; FVE =  0.56 (0.023 sec)
Step 13100: loss = 0.0803; FVE =  0.60 (0.025 sec)
Step 13150: loss = 0.0957; FVE =  0.52 (0.034 sec)
Step 13200: loss = 0.0844; FVE =  0.58 (0.038 sec)
early stop
Performance is:
squared (loss) = 0.1077; FVE = 0.463
r = 0.596
More training!
Step 1

...

Step 42700: loss = 0.0475; FVE =  0.76 (0.021 sec)
Step 42750: loss = 0.0471; FVE =  0.77 (0.022 sec)
Step 42800: loss = 0.0494; FVE =  0.75 (0.023 sec)
Step 42850: loss = 0.0459; FVE =  0.77 (0.023 sec)
Step 42900: loss = 0.0480; FVE =  0.76 (0.021 sec)
Step 42950: loss = 0.0479; FVE =  0.76 (0.038 sec)
Step 43000: loss = 0.0500; FVE =  0.75 (0.022 sec)
Step 43050: loss = 0.0545; FVE =  0.73 (0.023 sec)
Step 43100: loss = 0.0467; FVE =  0.77 (0.022 sec)
Step 43150: loss = 0.0500; FVE =  0.75 (0.021 sec)
Step 43200: loss = 0.0506; FVE =  0.75 (0.021 sec)
Step 43250: loss = 0.0495; FVE =  0.75 (0.021 sec)
Step 43300: loss = 0.0467; FVE =  0.77 (0.021 sec)
Step 43350: loss = 0.0486; FVE =  0.76 (0.022 sec)
Step 43400: loss = 0.0513; FVE =  0.74 (0.022 sec)
Step 43450: loss = 0.0463; FVE =  0.77 (0.020 sec)
Step 43500: loss = 0.0468; FVE =  0.77 (0.021 sec)
Step 43550: loss = 0.0510; FVE =  0.75 (0.037 sec)
Step 43600: loss = 0.0470; FVE =  0.77 (0.021 sec)
Step 43650: loss = 0.0446; FVE 

Step 50750: loss = 0.0449; FVE =  0.78 (0.022 sec)
Step 50800: loss = 0.0469; FVE =  0.77 (0.023 sec)
Step 50850: loss = 0.0460; FVE =  0.77 (0.021 sec)
Step 50900: loss = 0.0473; FVE =  0.76 (0.020 sec)
Step 50950: loss = 0.0432; FVE =  0.78 (0.025 sec)
Step 51000: loss = 0.0506; FVE =  0.75 (0.020 sec)
Step 51050: loss = 0.0421; FVE =  0.79 (0.022 sec)
Step 51100: loss = 0.0459; FVE =  0.77 (0.020 sec)
Step 51150: loss = 0.0448; FVE =  0.78 (0.023 sec)
Step 51200: loss = 0.0423; FVE =  0.79 (0.023 sec)
Step 51250: loss = 0.0450; FVE =  0.78 (0.041 sec)
Step 51300: loss = 0.0413; FVE =  0.79 (0.045 sec)
Step 51350: loss = 0.0469; FVE =  0.77 (0.021 sec)
Step 51400: loss = 0.0457; FVE =  0.77 (0.024 sec)
Step 51450: loss = 0.0434; FVE =  0.78 (0.021 sec)
Step 51500: loss = 0.0497; FVE =  0.75 (0.022 sec)
Step 51550: loss = 0.0429; FVE =  0.79 (0.021 sec)
Step 51600: loss = 0.0451; FVE =  0.78 (0.024 sec)
Step 51650: loss = 0.0466; FVE =  0.77 (0.027 sec)
Step 51700: loss = 0.0405; FVE 

Step 58800: loss = 0.0385; FVE =  0.81 (0.025 sec)
Step 58850: loss = 0.0375; FVE =  0.81 (0.024 sec)
Step 58900: loss = 0.0445; FVE =  0.78 (0.032 sec)
Step 58950: loss = 0.0444; FVE =  0.78 (0.024 sec)
Step 59000: loss = 0.0378; FVE =  0.81 (0.023 sec)
Step 59050: loss = 0.0411; FVE =  0.80 (0.021 sec)
Step 59100: loss = 0.0431; FVE =  0.79 (0.021 sec)
Step 59150: loss = 0.0417; FVE =  0.79 (0.038 sec)
Step 59200: loss = 0.0445; FVE =  0.78 (0.037 sec)
Step 59250: loss = 0.0451; FVE =  0.78 (0.021 sec)
Step 59300: loss = 0.0417; FVE =  0.79 (0.024 sec)
Step 59350: loss = 0.0411; FVE =  0.80 (0.020 sec)
Step 59400: loss = 0.0451; FVE =  0.78 (0.023 sec)
Step 59450: loss = 0.0427; FVE =  0.79 (0.022 sec)
Step 59500: loss = 0.0417; FVE =  0.79 (0.019 sec)
Step 59550: loss = 0.0425; FVE =  0.79 (0.034 sec)
Step 59600: loss = 0.0398; FVE =  0.80 (0.024 sec)
Step 59650: loss = 0.0413; FVE =  0.79 (0.023 sec)
Step 59700: loss = 0.0413; FVE =  0.79 (0.024 sec)
Step 59750: loss = 0.0415; FVE 

SystemExit: 

/home/ola-sammy/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
